In [18]:
from google.colab import drive
drive.mount('/content/drive')
import sys
import os
sys.path.append('/content/drive/My Drive/ENM_5320_Final_Project')
path = "/content/drive/My Drive/ENM_5320_Final_Project"
os.chdir(path)

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

import pickle as pkl
from pathlib import Path
from IPython.display import clear_output
import ipywidgets as widgets
from time import sleep
from tqdm import trange
import wandb

from FNO import FNO

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
def save_checkpoint(model, optimizer, epoch, name, checkpoint_dir='checkpoints'):
    Path(checkpoint_dir).mkdir(exist_ok=True)

    checkpoint = {
        'model_state': model.state_dict(),
        'optimizer_state': optimizer.state_dict(),
        'epoch': epoch
    }

    path = os.path.join(checkpoint_dir, name+f'_{epoch}.pt')
    torch.save(checkpoint, path)

def load_checkpoint(model, optimizer, checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state'])
    optimizer.load_state_dict(checkpoint['optimizer_state'])
    return checkpoint['epoch']

In [38]:
# global parameters
B = 15 # batch size
nx = 500 # npts
Nsols = 500

modes = 24
num_fourier_layers = 4
in_channels = 4
out_channels = 3
fourier_channels = 64
activation = nn.Tanh()
epochs = 10000

In [31]:
with open("./outputs_train.pkl", 'rb') as file:
    output_data = pkl.load(file)
len(output_data)

375

# Initialize Data Loader

In [34]:
class SodShockData(Dataset):
    def __init__(self, path, split: str = "train", normalize: bool = True):
        super(SodShockData, self).__init__()
        with open(Path(path) / f"inputs_{split}.pkl", 'rb') as file:
            self.input_data = pkl.load(file)

        with open(Path(path) / f"outputs_{split}.pkl", 'rb') as file:
            self.output_data = pkl.load(file)
        self.normalize = normalize
    def __len__(self):
        return len(self.input_data)

    def __getitem__(self, idx:int):
        inp = self.input_data[idx]
        x, rho, u, p = inp
        input_data = torch.stack((torch.as_tensor(x),torch.as_tensor(rho),
                                  torch.as_tensor(u),torch.as_tensor(p)), dim=-1)

        out = self.output_data[idx]
        rho, u, p = out
        output_data = torch.stack((torch.as_tensor(rho),torch.as_tensor(u),
                                   torch.as_tensor(p)), dim=-1)

        if self.normalize:
            input_mean = input_data.mean()
            input_std = input_data.std()
            if input_std <= 1e-10:
              input_std += 1e-6

            input_data = (input_data-input_mean)/input_std

            #print(input_data.mean())
            #print(input_data.std())
            #print(output_data.mean())
            #print(output_data.std())

            output_mean = output_data.mean()
            output_std = output_data.std()
            if output_std <= 1e-10:
              output_std += 1e-6
            output_data = (output_data-output_mean)/output_std

        return (input_data.float().to(device), output_data.float().to(device))

In [35]:
data_path = "/content/drive/My Drive/ENM_5320_Final_Project/"
train_dataset = SodShockData(
    path=data_path,
    split="train",
)

train_loader = DataLoader(
    train_dataset,
    batch_size=B,
    shuffle=True,
)
len(train_dataset)

375

In [36]:
for batch in train_loader:
    inputs, outputs = batch
    # print(outputs)
    break

In [37]:
def model_train(model, train_loader, optimizer, loss_fn, epochs):
    """
    Trains the given model and tracks the training process using wandb.

    Args:
        model: The model to train.
        train_loader: The data loader for the training data.
        optimizer: The optimizer to use.
        loss_fn: The loss function to use.
        epochs: The number of epochs to train for.
    """
    wandb.init(project="ENM_5320_Project", name="fno-training", reinit="create_new")

    # Log model configuration to wandb
    wandb.config.update({
        "learning_rate": optimizer.param_groups[0]['lr'],
        "epochs": epochs,
        "batch_size": train_loader.batch_size,
        "loss_function": type(loss_fn).__name__,
        # Add other relevant model parameters here
    })

    # Watch model for gradients and parameters logging
    wandb.watch(model, log="all", log_freq=100)

    # Training loop
    for epoch in trange(epochs):
        for batch in train_loader:
            inputs, outputs = batch
            pred = model(inputs)

            loss = loss_fn(pred, outputs)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if epoch % 20 == 0 or epoch == epochs:
            wandb.log({"train_loss": loss})
            if epoch % 100 == 0:
                print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss}')

    # Finish wandb run
    wandb.finish()
    save_checkpoint(model, optimizer, epoch, "fno")
    return model.state_dict(), optimizer.state_dict()

# Train Model

In [ ]:
# model initialization
model = FNO(modes=modes, num_fourier_layers=num_fourier_layers, in_channels=in_channels, fourier_channels=fourier_channels, out_channels=out_channels, activation=activation).to(device)

# initialize optimizer
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
l = nn.MSELoss()
#model_state, optimizer_state = model_train(model, train_loader, optimizer, l, epochs)
loss_log = []
for epoch in trange(epochs):
    for batch in train_loader:
        inputs, outputs = batch
        pred = model(inputs)


        loss = l(pred, outputs)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print(loss)
        #zero_grad
        #backwards
        #step
        # print(inputs.shape, outputs.shape)
    if epoch % 10 == 0 or epoch == epochs:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss}')
        loss_log.append(loss.detach().cpu().item())


  0%|          | 1/10000 [00:05<14:03:07,  5.06s/it]

Epoch [1/10000], Loss: 0.10201162844896317


  0%|          | 11/10000 [00:46<11:51:10,  4.27s/it]

Epoch [11/10000], Loss: 0.0063332365825772285


  0%|          | 21/10000 [01:28<12:25:47,  4.48s/it]

...

 95%|█████████▍| 9451/10000 [10:48:12<36:37,  4.00s/it]

Epoch [9451/10000], Loss: 7.019195618340746e-05


 95%|█████████▍| 9461/10000 [10:48:53<38:13,  4.26s/it]

Epoch [9461/10000], Loss: 0.0007632633787579834


 95%|█████████▍| 9465/10000 [10:49:10<37:43,  4.23s/it]

In [ ]:
from google.colab import files
import helper
checkpoint = {
        'model_state': model.state_dict(),
        'optimizer_state': optimizer.state_dict(),
        'epoch': epoch
    }

checkpt_file = "fno"+f'_{epoch}.pt'
torch.save(checkpoint, path+'/'+checkpt_file)
files.download(checkpt_file)


In [ ]:
x = np.zeros(100)
rho = np.zeros(100)
#torch.stack((torch.as_tensor(x), torch.as_tensor(rho)), dim=-1)

In [ ]:
def activation(name='tanh'):
    if name in ['tanh', 'Tanh']:
        return nn.Tanh()
    elif name in ['relu', 'ReLU']:
        return nn.ReLU(inplace=True)
    elif name in ['lrelu', 'LReLU']:
        return nn.LeakyReLU(inplace=True)
    elif name in ['sigmoid', 'Sigmoid']:
        return nn.Sigmoid()
    elif name in ['softplus', 'Softplus']:
        return nn.Softplus(beta=4)
    elif name in ['celu', 'CeLU']:
        return nn.CELU()
    elif name in ['elu']:
        return nn.ELU()
    elif name in ['SiLU']:
        return nn.SiLU()
    else:
        raise ValueError('Unknown activation function')

TypeError: 'int' object is not iterable

In [ ]:
# dataloader intialization

with open("sod_shock_dataset.pkl", "rb") as f:
    solutions = pkl.load(f)


# extract data

# train dataset

# test dataset

In [ ]:
for (init_cond, positions, regions, values) in solutions:
    x = values['x']
    rho = values['rho']
    u = values['u']
    p = values['p']
    break
rho.shape
x

array([0.        , 0.00200401, 0.00400802, 0.00601202, 0.00801603,
       0.01002004, 0.01202405, 0.01402806, 0.01603206, 0.01803607,
       0.02004008, 0.02204409, 0.0240481 , 0.0260521 , 0.02805611,
       0.03006012, 0.03206413, 0.03406814, 0.03607214, 0.03807615,
       0.04008016, 0.04208417, 0.04408818, 0.04609218, 0.04809619,
       0.0501002 , 0.05210421, 0.05410822, 0.05611222, 0.05811623,
       0.06012024, 0.06212425, 0.06412826, 0.06613226, 0.06813627,
       0.07014028, 0.07214429, 0.0741483 , 0.0761523 , 0.07815631,
       0.08016032, 0.08216433, 0.08416834, 0.08617234, 0.08817635,
       0.09018036, 0.09218437, 0.09418838, 0.09619238, 0.09819639,
       0.1002004 , 0.10220441, 0.10420842, 0.10621242, 0.10821643,
       0.11022044, 0.11222445, 0.11422846, 0.11623246, 0.11823647,
       0.12024048, 0.12224449, 0.1242485 , 0.12625251, 0.12825651,
       0.13026052, 0.13226453, 0.13426854, 0.13627255, 0.13827655,
       0.14028056, 0.14228457, 0.14428858, 0.14629259, 0.14829

In [ ]:
# optimizer

In [ ]:
# loss function

In [ ]:
# training loop

#